In [1]:
# write all code in one cell 

#========================Load data=========================
import numpy as np
import pandas as pd

train_data_source = 'C:\\Users\\Admin\\Dropbox\\Spring 2019\\Masters project\\Project\\yelp_train.csv'
test_data_source = 'C:\\Users\\Admin\\Dropbox\\Spring 2019\\Masters project\\Project\\yelp_test.csv'

train_df = pd.read_csv(train_data_source)
test_df = pd.read_csv(test_data_source)

In [2]:
# to change use .astype() 
test_df['text'] = test_df.text.astype(str)
train_df['text'] = train_df.text.astype(str)

In [3]:
# train_df.head()
# test_df.dtypes

In [4]:
# convert string to lower case 
train_texts = train_df['text'].values 
train_texts = [s.lower() for s in train_texts] 

test_texts = test_df['text'].values 
test_texts = [s.lower() for s in test_texts] 

In [5]:
#=======================Convert string to index================
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)

# Convert string to index 
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

C:\Users\Admin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
# Padding
train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
test_data = pad_sequences(test_texts, maxlen=1014, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')

In [7]:
#=======================Get classes================
train_classes = train_df['label'].values
train_class_list = [x-1 for x in train_classes]

test_classes = test_df['label'].values
test_class_list = [x-1 for x in test_classes]

from keras.utils import to_categorical
train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)

In [8]:
print(tk.word_index)

{'UNK': 1, ' ': 2, 'e': 3, 't': 4, 'a': 5, 'o': 6, 'i': 7, 'n': 8, 's': 9, 'r': 10, 'h': 11, 'd': 12, 'l': 13, 'u': 14, 'c': 15, 'w': 16, 'm': 17, 'y': 18, 'f': 19, 'g': 20, 'p': 21, '.': 22, 'b': 23, 'v': 24, 'k': 25, ',': 26, '\\': 27, "'": 28, '!': 29, 'x': 30, 'j': 31, '-': 32, 'z': 33, '0': 34, '"': 35, ')': 36, '(': 37, 'q': 38, '1': 39, '2': 40, '?': 41, '5': 42, '$': 43, '3': 44, ':': 45, '4': 46, '/': 47, '9': 48, '&': 49, '8': 50, '6': 51, '7': 52, '*': 53, ';': 54, '+': 55, '%': 56, '=': 57, '#': 58, '_': 59, '@': 60, '~': 61, ']': 62, '[': 63, '^': 64, '|': 65, '{': 66, '`': 67}


In [9]:
vocab_size=len(tk.word_index)
vocab_size

67

In [10]:
embedding_weights=[]

In [11]:
embedding_weights.append(np.zeros(vocab_size))

In [12]:
for char, i in tk.word_index.items(): # from index 1 to 70
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)

In [13]:
print(embedding_weights.shape) # first row all 0 for PAD, 69 char, last row for UNK
embedding_weights

(68, 67)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [14]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [15]:
# parameter 
input_size = 1014
# vocab_size = 67
embedding_size = 67
conv_layers = [[256, 7, 3], 
               [256, 7, 3], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 2
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

In [16]:
# Embedding layer Initialization
embedding_layer = Embedding(vocab_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

In [17]:
# Model 

# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding 
x = embedding_layer(inputs)
# Conv 
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x) 
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x) # Final shape=(None, 34, 256)
x = Flatten()(x) # (None, 8704)
# Fully connected layers 
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x) # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) # Adam, categorical_crossentropy
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1014)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1014, 67)          4556      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1008, 256)         120320    
_________________________________________________________________
activation_1 (Activation)    (None, 1008, 256)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 336, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 330, 256)          459008    
_________________________________________________________________
activation_2 (Activation)    (None, 330, 256)          0         
__________

In [18]:
# 1000 training samples and 100 testing samples
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices][:]
y_train = train_classes[indices][:]

x_test = test_data[:]
y_test = test_classes[:]

In [19]:
# Training
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=6,
          verbose=2)

Train on 14000 samples, validate on 3800 samples
Epoch 1/6
 - 832s - loss: 0.6933 - acc: 0.5522 - val_loss: 0.6866 - val_acc: 0.5682
Epoch 2/6
 - 827s - loss: 0.6206 - acc: 0.6511 - val_loss: 0.5166 - val_acc: 0.7529
Epoch 3/6
 - 825s - loss: 0.4964 - acc: 0.7660 - val_loss: 0.3725 - val_acc: 0.8424
Epoch 4/6
 - 824s - loss: 0.3384 - acc: 0.8561 - val_loss: 0.2945 - val_acc: 0.8771
Epoch 5/6
 - 824s - loss: 0.2473 - acc: 0.8990 - val_loss: 0.2691 - val_acc: 0.8926
Epoch 6/6
 - 824s - loss: 0.2038 - acc: 0.9227 - val_loss: 0.3143 - val_acc: 0.8895


In [20]:
model.save("C:\\Users\\Admin\\Dropbox\\Spring 2019\\Masters project\\Project\\my_model_yelp.h5")